In [109]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests as rq
from urllib.parse import urlencode

%matplotlib inline

In [52]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
customer_data = 'https://disk.yandex.ru/d/Ta-weYfGM5CHjg'

# Получаем загрузочную ссылку
download_url_customer_data = rq.get(base_url + urlencode(dict(public_key=customer_data))).json()['href']
download_response = rq.get(download_url_customer_data)

In [55]:
customers_df = pd.read_csv(download_url_customer_data, parse_dates=['date'])

### Описание данных 

- **`date`** – дата совершения события

- **`event`** - событие
    + *app_install – установка приложения*
    + *app_start – открыть приложения*
    + *registration – зарегистрироваться*
    + *search – перейти на страницу поиска товаров (каталог)*
    + *open_item – открыть товар*
    + *choose_item – отправить товар в корзину*
    + *tap_basket – перейти в корзину*
    + *purchase – подтверждение покупки*
    
- **`purchase_sum`** – стоимость покупки (при совершении события ‘purchase’)

- **`os_name`** – платформа пользователя

- **`device_id`** – идентификатор устройства пользователя

- **`gender`** – пол пользователя

- **`city`** – город пользователя

- **`urm_source`** – канал, с которого пришел пользователь
    + *yandex-direct – Яндекс директ*
    + *google_ads – реклама в Google*
    + *vk_ads – реклама в ВК*
    + *instagram_ads – реклама в instagram*
    + *facebook_ads – реклама в facebook*
    + *referral – акция «приведи друга»*
    + ***Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы***  

In [56]:
customers_df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [7]:
customers_df.shape

(2747968, 8)

In [40]:
customers_df.dtypes

date             object
event            object
purchase_sum    float64
os_name          object
device_id         int64
gender           object
city             object
utm_source       object
dtype: object

In [91]:
customers_df.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [90]:
f'date_time from {customers_df.date.min()} to {customers_df.date.max()}'

'date_time from 2020-01-01 00:00:00 to 2020-03-31 00:00:00'

In [92]:
customers_df.os_name.unique()

array(['android', 'ios'], dtype=object)

In [99]:
customers_df.event.value_counts().sort_values(ascending=False)

app_start      748705
search         708639
choose_item    538669
tap_basket     377665
app_install    154597
purchase       141383
register        78310
Name: event, dtype: int64

In [100]:
customers_df.city.value_counts().sort_values(ascending=False)

Moscow              1491481
Saint-Petersburg    1256487
Name: city, dtype: int64

In [101]:
customers_df.utm_source.value_counts().sort_values(ascending=False)

-                610458
vk_ads           418456
yandex-direct    390232
instagram_ads    382891
google_ads       374257
facebook_ads     336953
referal          234721
Name: utm_source, dtype: int64

In [108]:
f'number of rows:{customers_df.device_id.count()}, and number of unique device_ids(users):{customers_df.device_id.nunique()}'

'number of rows:2747968, and number of unique device_ids(users):190884'

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [116]:
f'avg num of actions per customer:{customers_df.device_id.count()/customers_df.device_id.nunique()}'

'avg num of actions per customer:14.396010142285368'

In [140]:
customers_df.loc[customers_df.date.dt.month == 2].device_id.nunique()

75032

In [145]:
customers_df.loc[(customers_df.date.dt.month == 1) & (customers_df.event == 'app_install')].event.count()

80297